# Corrections des zones 9xx

Source : Fichier http://ge.rero.ch/rero/tasks/task_coca_T186.1.6/fields_9xx_ge.txt

But :  filtrer le contenu pour isoler les erreurs

Moyens : découper le fichier en 4 parties, 902, 932, 982 et 992

- Zone 902 : https://www.rero.ch/page.php?section=zone&pageid=90x
    - pas d'indicateurs (deux espaces vides après le chiffre, 9021 c'est Faux).
    - Sous-zones : a (obligatoire, contient que du texte) b c n (sensible à la case)
    - Valeurs admis
        - a : https://www.rero.ch/page.php?section=zone&pageid=902
        - si b (année en cours) : AAAA/MM
        - si c (moention du chapitre pour le tri) : chiffres avec point seulement

- zone 932 : https://www.rero.ch/page.php?section=zone&pageid=93x
    - pas d'indicateurs (deux espaces vides après le chiffre, 9321 c'est Faux).
    - Sous-zones : a (obligatoire)
    - Valeurs admis
        - a : valeurs multiples possibles (séparateur ","), contient lettres et chiffres seulement et doit commencer par des lettres (gevbaa, gevbge, gevbmu, gevcjb). Exemples : 
            - 32 a gevcjb481100
            - 932 a gevbge0050609, gevbge0050610


- zone 982 : https://www.rero.ch/page.php?section=zone&pageid=98x
    - pas d'indicateurs (deux espaces vides après le chiffre, 9821 c'est Faux).
    - Sous-zones : 2 (obligatoire, au début), a
    - Valeurs admis
        - 2 : https://www.rero.ch/page.php?section=zone&pageid=982

- zone 992 : https://www.rero.ch/page.php?section=zone&pageid=99x
    - pas d'indicateurs (deux espaces vides après le chiffre, 9921 c'est Faux).
    - Sous-zones : a x (obligatoirs)
    - Valeurs admis
        - a (cote) : chiffres, lettres et un seul tiret
    
## Traitement

1. répetition des informations sure les lignes du bas pour avoir un CSV complet
1. découpage de la zone pour isoler les 2 indicateurs
1. ajouter une colonne pour l'erreur (0/1)
1. ajouter une colonne pour le type d'erreur (codes [I1, I2, SZa, SZx, SZ2])
1. appliquer les conditions pour ajouter les valeurs d'erreur

Fichier final
```
zone | indicateur1 | indicateur2 | souszone | valeur | occurences | erreur | erreur_type
902	 	 	0	2	1
902	 	 	0	6	1
902	 	 	1	4	1
...
902	 	 	2	004/04	1
```


In [1]:
import pandas as pd
df = pd.read_csv('sample.txt', delimiter='\t', header=None, names=['zoneorig', 'souszone', 'valeur', 'occurences'],
                 dtype={'zoneorig' : str, 'souszone' : str, 'valeur' : str, 'occurences' : float})
df = df.fillna(method='ffill')
df

,zoneorig,souszone,valeur,occurences
0,902,NaN,NaN,NaN
1,902,0,NaN,NaN
2,902,0,2,1.0
3,902,0,6,1.0
4,902,1,6,1.0
5,902,1,4,1.0
6,902,1,55,1.0
7,902,1,998/03,1.0
8,902,1,998/07,1.0
9,902,1,998/10,1.0


In [2]:
df['zone'] = df['zoneorig'].str[0:3]
df['indicateur1'] = df['zoneorig'].str[3] 
df['indicateur2'] = df['zoneorig'].str[4] 
df

,zoneorig,souszone,valeur,occurences,zone,indicateur1,indicateur2
0,902,NaN,NaN,NaN,902,,
1,902,0,NaN,NaN,902,,
2,902,0,2,1.0,902,,
3,902,0,6,1.0,902,,
4,902,1,6,1.0,902,,
5,902,1,4,1.0,902,,
6,902,1,55,1.0,902,,
7,902,1,998/03,1.0,902,,
8,902,1,998/07,1.0,902,,
9,902,1,998/10,1.0,902,,


In [3]:
# comptage des lignes par zone
df['zone'].value_counts()

982    492
992    314
902    150
932     44
Name: zone, dtype: int64

In [4]:
#ajout des champs d'erreur
df['erreur_i1'] = 0
df['erreur_i2'] = 0
df['erreur_souszone'] = 0
df['erreur_valeur'] = 0

In [5]:
# erreurs communs : il ne doit pas y avoir d'indicateurs
df.loc[(df['indicateur1'] != ' '), 'erreur_i1'] = 1
df.loc[(df['indicateur2'] != ' '), 'erreur_i2'] = 1

In [6]:
df.loc[df['erreur_i1'] == 1]

,zoneorig,souszone,valeur,occurences,zone,indicateur1,indicateur2,erreur_i1,erreur_i2,erreur_souszone,erreur_valeur
147,9021,z,A Joyc 8*GLAS Thi cof,1.0,902,1,,1,0,0,0
148,9021,a,A Joyc 8*GLAS Thi cof,1.0,902,1,,1,0,0,0
149,9021,a,gedoc,1.0,902,1,,1,0,0,0
191,9321,a,gevbge0760258,1.0,932,1,,1,0,0,0
192,9321,a,gevbge0760258,1.0,932,1,,1,0,0,0
193,9321,a,gevcjb300821,1.0,932,1,,1,0,0,0
681,9822,x,ge/vbpu/a/97/755,1.0,982,2,,1,0,0,0
682,9822,2,ge/vbpu/a/97/755,1.0,982,2,,1,0,0,0
683,9822,2,cdu-geusst,1.0,982,2,,1,0,0,0
684,9822,a,cdu-geusst,1.0,982,2,,1,0,0,0


In [7]:
# 902
# Sous-zones : a (obligatoire, contient que du texte) b c n (sensible à la case)
df.loc[((df['zone'] == '902') & (df['souszone'] != 'a') & (df['souszone'] != 'b') & (df['souszone'] != 'c') & (df['souszone'] != 'n') & (df['souszone'].notnull())), 'erreur_souszone'] = 1

In [8]:
# Valeurs admis
#    a : https://www.rero.ch/page.php?section=zone&pageid=902
#    si b (année des nouvelles acquisitions) : AAAA/MM (AAAA >= 1998 & AAAA <> 2018)
#    si c (moention du chapitre pour le tri) : chiffres avec point seulement
mes902admis = ['airbau', 'airbie', 'aircla', 'airdup', 'airfer', 'airise', 'airlpe', 'airmdp', 'airmdpal', 'airuld', 'arch', 'baap', 'baas', 'cathor19', 'chaix', 'cinemed', 'deplem', 'doncollart', 'donomshim', 'elivgeheid', 'elivgeheds', 'elivgeucds', 'elivgeucmu', 'elivgeucmusem', 'elivgeufpe', 'elivgewcc', 'epergevbaa', 'epergewcc', 'fondsdubois', 'fondsdunand', 'fondscandaux', 'fondskasser', 'fondslivelihood', 'fondscortney', 'fondsrappard', 'fondssouvarine', 'frapin', 'fsantin', 'ge16', 'geaut', 'geav', 'gecaeg', 'gecjpja', 'gecjpj1', 'gecjpj2', 'gecjpj3', 'gecjpj5', 'gecjpj7', 'gecjpj8', 'gecjpj10', 'gecjpj12', 'gecjpj13', 'gecjpj14', 'gecjpj18', 'gecjpj23', 'gecjpj24', 'gecjpj25', 'gecjpj26', 'gecjpj27', 'gecjpj28', 'gecjpj29', 'gedoc', 'gefili', 'gefoha', 'gehead', 'geheds', 'geheid', 'geheg-', 'geiess', 'gelocalrecord', 'geprêt', 'gesesfkoenig', 'geubfd', 'geubib', 'geucdbf', 'geucds', 'geuceje', 'geuchr', 'geucig', 'geucmu', 'geucui', 'geuels', 'geueti', 'geufpe', 'geuieg', 'geuise', 'geul-l', 'geulal', 'geulan', 'geular', 'geulcf', 'geulch', 'geulco', 'geules', 'geulfr', 'geulgm', 'geulha', 'geulhg', 'geulit', 'geulja', 'geullc', 'geullg', 'geulmu', 'geulne', 'geulph', 'geulpo', 'geulro', 'geulru', 'geulsa', 'geumem', 'geusba', 'geuscr', 'geuses', 'geusob', 'geuspa', 'geussa', 'geussc', 'geussm', 'geussp', 'geusst', 'geuth-', 'gevbaa', 'gevbaaarchives', 'gevbibhodler', 'gevbmu', 'gevbmuthem', 'gevbge', 'gevbgedl', 'gevcjb', 'gevdochodler', 'gevdocmoos', 'geveth', 'gevfgalb', 'gevfim', 'gevfp', 'gevfpop', 'gevfpub', 'gevfrst', 'gevfseg', 'gevfvw', 'gevmad', 'gevmah', 'gevmar', 'gevmcd', 'gevmce', 'gevmci', 'gevmcn', 'gevmda', 'gevmhe', 'gevmhn', 'gevmhs', 'gevmla', 'gevmrt', 'gevmsec', 'gewcc-', 'glfbe', 'padmin', 'pubbos', 'pubcoe', 'pubhei', 'pubheid', 'pubiued', 'recatge01', 'recatge02', 'recatge03', 'recatge04', 'recatge05', 'recatheidcdu', 'recatheidejp', 'recatheidsdn', 'riv19', 'rivrest', 'silg', 'suitebge', 'tjg', 'tjg86', 'tpseg']
df.loc[((df['zone'] == '902') & (df['souszone'] == 'a')) & (~(df['valeur'].isin(mes902admis))), 'erreur_valeur'] = 1

In [9]:
df.loc[((df['zone'] == '902') & (df['souszone'] == 'b'))]['valeur'].value_counts()

Series([], Name: valeur, dtype: int64)

In [10]:
df.loc[((df['zone'] == '902') & (df['souszone'] == 'b')) & (df['valeur'].str.match('[0-9][0-9][0-9][0-9]\/[0-9][0-9]'))]

,zoneorig,souszone,valeur,occurences,zone,indicateur1,indicateur2,erreur_i1,erreur_i2,erreur_souszone,erreur_valeur


In [11]:
# extraction des b
df['b_date'] = df['valeur'].str[0:4]
df['b_mois'] = df['valeur'].str[4:6]

In [12]:
# si b (année en cours) : AAAA/MM
# liste de valeurs (AAAA >= 1998 & AAAA <> 2018)
mesdates = ['1998', '1999', '2000', '2001', '2002', '2003', '2004', '2005', '2006', '2007', '2008', '2009', '2010', '2011', '2012', '2013', '2014', '2015', '2016', '2017', '2018']
mesmois = ['01', '02', '03', '04', '05', '06', '07', '08', '09', '10', '11', '12']
df.loc[((df['zone'] == '902') & (df['souszone'] == 'b')) & (~(df['b_date'].isin(mesdates))), 'erreur_valeur'] = 1
df.loc[((df['zone'] == '902') & (df['souszone'] == 'b')) & (~(df['b_mois'].isin(mesmois))), 'erreur_valeur'] = 1

In [13]:
# si c (moention du chapitre pour le tri) : chiffres avec point seulement [0-9]+{1-3} ou [0-9]+{1-3}\.[0-9]+{1-3}
df.loc[((df['zone'] == '902') & (df['souszone'] == 'c'))]['valeur'].value_counts()

Series([], Name: valeur, dtype: int64)

In [14]:
df.loc[((df['zone'] == '902') & (df['souszone'] == 'c') & (df['valeur'].str.contains('^([0-9]{1,3}|[0-9]{1,3}\.[0-9]{1,3})$', regex=True)))]

C:\Users\iriarte\AppData\Local\Continuum\anaconda3\lib\site-packages\ipykernel_launcher.py:1: UserWarning: This pattern has match groups. To actually get the groups, use str.extract.
  """Entry point for launching an IPython kernel.


,zoneorig,souszone,valeur,occurences,zone,indicateur1,indicateur2,erreur_i1,erreur_i2,erreur_souszone,erreur_valeur,b_date,b_mois


In [15]:
df.loc[((df['zone'] == '902') & (df['souszone'] == 'c')), 'erreur_valeur'] = 1
df.loc[((df['zone'] == '902') & (df['souszone'] == 'c') & (df['valeur'].str.contains('^([0-9]{1,3}|[0-9]{1,3}\.[0-9]{1,3})$', regex=True))), 'erreur_valeur'] = 0

C:\Users\iriarte\AppData\Local\Continuum\anaconda3\lib\site-packages\ipykernel_launcher.py:2: UserWarning: This pattern has match groups. To actually get the groups, use str.extract.
  


In [16]:
# supprimer les colonnes calculées
del df['b_date']
del df['b_mois']

In [17]:
# check des erreurs
df.loc[(df['zone'] == '902') & (df['souszone'] == 'c') & (df['erreur_valeur'] == 1)]

,zoneorig,souszone,valeur,occurences,zone,indicateur1,indicateur2,erreur_i1,erreur_i2,erreur_souszone,erreur_valeur


In [18]:
# 932    Valeurs admis
#        a : valeurs multiples possibles (séparateur ","), 

# test de souszones
df.loc[df['zone'] == '932']['souszone'].value_counts()

a    41
x     3
Name: souszone, dtype: int64

In [19]:
# seulement a est admis
df.loc[((df['zone'] == '932') & (df['souszone'] != 'a')), 'erreur_souszone'] = 1

In [20]:
# test de valeurs multiples
df.loc[(df['zone'] == '932') & (df['valeur'].str.contains(', '))]

,zoneorig,souszone,valeur,occurences,zone,indicateur1,indicateur2,erreur_i1,erreur_i2,erreur_souszone,erreur_valeur


In [21]:
#  contient lettres et chiffres seulement et doit commencer par des lettres (gevbaa, gevbge, gevbmu, gevcjb).
#  Exemples : 932 a gevcjb481100 ; 932 a gevbge0050609, gevbge0050610
df.loc[((df['zone'] == '932') & (df['souszone'] == 'a') & (df['valeur'].str.contains('^gev(baa|bge|bmu|cjb)[0-9]+$', regex=True)))]


C:\Users\iriarte\AppData\Local\Continuum\anaconda3\lib\site-packages\ipykernel_launcher.py:3: UserWarning: This pattern has match groups. To actually get the groups, use str.extract.
  This is separate from the ipykernel package so we can avoid doing imports until


,zoneorig,souszone,valeur,occurences,zone,indicateur1,indicateur2,erreur_i1,erreur_i2,erreur_souszone,erreur_valeur
176,932,a,gevcjb481096,1.0,932,,,0,0,0,0
177,932,a,gevcjb481098,1.0,932,,,0,0,0,0
178,932,a,gevcjb481100,1.0,932,,,0,0,0,0
189,932 0,a,gevbaa060261,1.0,932,,0,0,1,0,0
190,932 0,a,gevbge0760258,1.0,932,,0,0,1,0,0
191,9321,a,gevbge0760258,1.0,932,1,,1,0,0,0
192,9321,a,gevbge0760258,1.0,932,1,,1,0,0,0
193,9321,a,gevcjb300821,1.0,932,1,,1,0,0,0


In [22]:
df.loc[((df['zone'] == '932') & (df['souszone'] == 'a')), 'erreur_valeur'] = 1
df.loc[((df['zone'] == '932') & (df['souszone'] == 'a') & (df['valeur'].str.contains('^gev(baa|bge|bmu|cjb)[0-9]+$', regex=True))), 'erreur_valeur'] = 0

C:\Users\iriarte\AppData\Local\Continuum\anaconda3\lib\site-packages\ipykernel_launcher.py:2: UserWarning: This pattern has match groups. To actually get the groups, use str.extract.
  


In [23]:
# exclure par défaut les lignes avec valeurs multiple
df.loc[((df['zone'] == '932') & (df['souszone'] == 'a') & (df['valeur'].str.contains(', ', regex=False))), 'erreur_valeur'] = 0

In [24]:
df_mv = df.loc[((df['zone'] == '932') & (df['souszone'] == 'a') & (df['valeur'].str.contains(', ', regex=False)))]

In [25]:
# boucle sur les lignes et sur les valeurs de la liste
df_mv['valeurt'] = df_mv['valeur'].str.split(', ')
import re
for index, row in df_mv.iterrows():
    for i in range(len(row['valeurt'])):
        print(index, i, row['valeurt'][i], row['erreur_valeur'])
        if (re.search(r'^gev(baa|bge|bmu|cjb)[0-9]+$', row['valeurt'][i])):
            print ('OK')
        else:
            print ('FAUX')
            df_mv.loc[index, 'erreur_valeur'] = 1
            df.loc[index, 'erreur_valeur'] = 1

In [26]:
# check des erreurs
df.loc[(df['zone'] == '932') & (df['souszone'] == 'a') & (df['erreur_valeur'] == 1)]

,zoneorig,souszone,valeur,occurences,zone,indicateur1,indicateur2,erreur_i1,erreur_i2,erreur_souszone,erreur_valeur
150,932,a,gedoc,1.0,932,,,0,0,0,1
151,932,a,gedoc,1.0,932,,,0,0,0,1
152,932,a,0705321,1.0,932,,,0,0,0,1
153,932,a,1064024494,1.0,932,,,0,0,0,1
154,932,a,1064030131,1.0,932,,,0,0,0,1
155,932,a,1064038756,1.0,932,,,0,0,0,1
156,932,a,1064048279,1.0,932,,,0,0,0,1
157,932,a,1064078845,1.0,932,,,0,0,0,1
158,932,a,1064093839,1.0,932,,,0,0,0,1
159,932,a,1064111378,1.0,932,,,0,0,0,1


In [27]:
# zone 982 : https://www.rero.ch/page.php?section=zone&pageid=98x
#    pas d'indicateurs (deux espaces vides après le chiffre, 9821 c'est Faux).
#    Sous-zones : 2 (obligatoire, au début), a

# test de souszones
df.loc[df['zone'] == '982']['souszone'].value_counts()

d    177
2    102
e     94
a     82
x     17
c      7
b      5
j      2
f      2
A      2
s      2
Name: souszone, dtype: int64

In [28]:
# seulement a est admis
df.loc[((df['zone'] == '982') & (df['souszone'] != 'a') & (df['souszone'] != '2')), 'erreur_souszone'] = 1

In [29]:
# test de valeurs en $2
df.loc[((df['zone'] == '982') & (df['souszone'] == '2'))]['valeur'].value_counts()

Ge-ulsh                           1
DEW-MEUM                          1
nlm-geuhim                        1
959                               1
cdu-nencm                         1
982                               1
WY 11 BOY                         1
7.071 GILL/5 (DVD)                1
519.5 HUC                         1
cdu geussc                        1
Af MAN and mei                    1
D.4.0 QUI Ed.2                    1
cdu-sys-geusc                     1
Db D lan ris                      1
ams-geussm                        1
HEIBR 301+1                       1
piaget                            1
650.1 MONb                        1
pcdm 3.35 PERG                    1
004.678 COL                       1
CDU-SYS-GEUIEG                    1
398.2 BET                         1
gevcjb300821                      1
ams-geudbu                        1
pscm                              1
cdu geulja                        1
nlm-geucds                        1
EG:Coll*MIFAO 80            

In [30]:
#    Valeurs admis en $2 : https://www.rero.ch/page.php?section=zone&pageid=982
mes982admis = ['acm-geucui', 'air', 'ams-geussm', 'cdu-fili', 'cdu-geheds', 'cdu-gehei', 'cdu-geufpe', 'cdu-geuieg', 'cdu-geul-l', 'cdu-geulco', 'cdu-geulja', 'cdu-geussc', 'cdu-geusst', 'cdu-geuth', 'cdu-geuth-g', 'cdu-gevcjb', 'cdu-gevmhs', 'cdu-head', 'cdu-iued', 'cdu-sys-gehei', 'cdu-sys-geuieg', 'cdu-sys-geussc', 'dew-geheg-', 'dew-geies', 'dew-geubib', 'dew-geuise', 'dew-geulcf', 'dew-geuscr', 'dew-geuses', 'dew-gevbaa', 'dew-gevbge', 'dew-gewcc', 'dew-meum', 'dew-sys-geubib', 'dew-sys-geuscr', 'dr-art', 'dr-gecjpj', 'dr-geubfd', 'dr-sys-gecjpj', 'eti1', 'eti2', 'eurovoc-geuig', 'eurovoc-sys', 'fili-mc', 'fili-tl', 'ge-baap', 'ge-baav', 'ge-biba', 'ge-heid', 'ge-heid-nc', 'ge-ies-', 'ge-obs', 'ge-piag', 'ge-sys-ph', 'ge-ulal', 'ge-ulam', 'ge-ulan', 'ge-ular', 'ge-ulch', 'ge-ules', 'ge-ulfm', 'ge-ulfr', 'ge-ulgm', 'ge-ulha', 'ge-ulhg', 'ge-ulit', 'ge-ullc', 'ge-ullg', 'ge-ulmu', 'ge-ulph', 'ge-ulpo', 'ge-ulro', 'ge-ulru', 'ge-ulsa', 'ge-ulsh', 'ge-usob', 'ge-ussa', 'ge-ussm', 'ge-ussp', 'ge-usst', 'ge-vbaa', 'ge-vbge', 'ge-vimv', 'ge-vmhn', 'genava', 'genavr', 'hardt', 'head-artiste', 'head-voc', 'heid', 'iued', 'jv', 'meum', 'nb', 'nlm-geucds', 'nlm-geucmu', 'nlm-geuhim', 'pcdm', 'pcdm-sys', 'piaget', 'twse', 'wccsh']
df.loc[((df['zone'] == '982') & (df['souszone'] == '2')) & (~(df['valeur'].isin(mes982admis))), 'erreur_valeur'] = 1

In [31]:
# zone 992 : https://www.rero.ch/page.php?section=zone&pageid=99x
#     Sous-zones : $a $x (obligatoires)

# test de souszones
df.loc[df['zone'] == '992']['souszone'].value_counts()

x    158
p     58
a     42
t      8
2      7
b      6
1      6
c      5
I      4
9      3
l      3
z      2
3      2
X      2
F      2
V      2
q      2
v      2
Name: souszone, dtype: int64

In [32]:
# seulement a et x sont admis
df.loc[((df['zone'] == '992') & (df['souszone'] != 'a') & (df['souszone'] != 'x')), 'erreur_souszone'] = 1

In [33]:
# test de valeurs en $a
df.loc[((df['zone'] == '992') & (df['souszone'] == 'a'))]['valeur'].value_counts()

/BGE Btm 14842                  1
شعراء الرابطة القلمية           1
‬BAA BR 14473                   1
wcc DM2001/10                   1
v                               1
x                               1
ÏHR LBR 665                     1
‬BAA Q 693                      1
'030 REY                        1
ËT AF 1450                      1
àa BGE Sg 2251                  1
‬DBUH 729                       1
/BGE Ib 4622                    1
/2013/263298.1                  1
528.9                           1
0.1 BARB                        1
‬BAA BR 14622                   1
-                               1
---                             1
‬BAA BR 14417                   1
‬BAA Q 543/1-2                  1
0.1 BOHE                        1
0. 674 AASA                     1
----                            1
‬BAA BR 14525                   1
‬BAA BR 14743                   1
‬‬159.955 (07) GOL.3            1
0.1 AVEN                        1
‬BAA 1738                       1
...           

In [34]:
# Valeurs admis en $a (cote) : chiffres, lettres et un seul tiret
df.loc[((df['zone'] == '992') & (df['souszone'] == 'a')), 'erreur_valeur'] = 1
df.loc[((df['zone'] == '992') & (df['souszone'] == 'a') & (df['valeur'].str.contains('^[0-9A-Za-z \-]+$', regex=True))), 'erreur_valeur'] = 0

In [35]:
# calculer erreur total
df_errors = df.loc[(df['erreur_i1'] == 1) | (df['erreur_i2'] == 1) | (df['erreur_souszone'] == 1) | (df['erreur_valeur'] == 1)]
df_errors

,zoneorig,souszone,valeur,occurences,zone,indicateur1,indicateur2,erreur_i1,erreur_i2,erreur_souszone,erreur_valeur
1,902,0,NaN,NaN,902,,,0,0,1,0
2,902,0,2,1.0,902,,,0,0,1,0
3,902,0,6,1.0,902,,,0,0,1,0
4,902,1,6,1.0,902,,,0,0,1,0
5,902,1,4,1.0,902,,,0,0,1,0
6,902,1,55,1.0,902,,,0,0,1,0
7,902,1,998/03,1.0,902,,,0,0,1,0
8,902,1,998/07,1.0,902,,,0,0,1,0
9,902,1,998/10,1.0,902,,,0,0,1,0
10,902,1,999/09,1.0,902,,,0,0,1,0


In [36]:
# nmbre d'erreurs
df_errors[['erreur_i1', 'erreur_i2', 'erreur_souszone', 'erreur_valeur']].sum()

erreur_i1           11
erreur_i2            4
erreur_souszone    541
erreur_valeur      156
dtype: int64

In [37]:
# exporter en csv
df_errors[['zone', 'indicateur1', 'indicateur2', 'souszone', 'valeur', 'occurences', 'erreur_i1', 'erreur_i2', 'erreur_souszone', 'erreur_valeur']].to_csv('sample_erreurs.txt', sep='\t', encoding='utf-8', index=False)